# Netflix Machine Learning - Modeling

In [1]:
# import libraries
import math
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
# import encoded dataset
data = pd.read_csv(r"C:\Users\Shinemet\Ironhack\Projects\Project7\Work_files\data_clean_encoded.csv")

In [3]:
# check data
data.head()

year  kind  rating     vote  runtime  genre1  country1  language1  cast1  \
0  2003.0     2     7.7    474.0     50.0       6        62         16    867   
1  1994.0     7     8.1     18.0     83.0       8        62         16   3627   
2  1992.0     1     5.5     93.0     95.0       0        37         33   2513   
3  2004.0     1     5.3  13432.0     60.0       6        62         16   3047   
4  1991.0     1     4.5   2177.0     96.0       4        62         16    639   

   cast2  director1  writer1  
0   4968       4121     3319  
1    438       3672     5036  
2   1667       4077     2610  
3   1545       4022     4892  
4   3350       1124     2893

### Data prep for modeling

In [4]:
# convert target variable to process models
data.rating = data.rating.astype(int)
data.head()

year  kind  rating     vote  runtime  genre1  country1  language1  cast1  \
0  2003.0     2       7    474.0     50.0       6        62         16    867   
1  1994.0     7       8     18.0     83.0       8        62         16   3627   
2  1992.0     1       5     93.0     95.0       0        37         33   2513   
3  2004.0     1       5  13432.0     60.0       6        62         16   3047   
4  1991.0     1       4   2177.0     96.0       4        62         16    639   

   cast2  director1  writer1  
0   4968       4121     3319  
1    438       3672     5036  
2   1667       4077     2610  
3   1545       4022     4892  
4   3350       1124     2893

In [5]:
# split dataset into independent and dependent variables
X = data.drop('rating', axis=1)
y = data['rating']

In [6]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
STOP

### Models to import

In [7]:
# import models
from sklearn import ensemble, datasets, tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

### Feature Selection

In [8]:
# Find the optimal number of features with SFM
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

SFM = SelectFromModel(estimator=RandomForestClassifier())
s = SFM.fit(X, y)

n_features = s.transform(X).shape[1]
n_features

7

In [9]:
# Get feature names 
feature_idx = s.get_support()
feature_name = X.columns[feature_idx]
feature_name

Index(['year', 'vote', 'runtime', 'cast1', 'cast2', 'director1', 'writer1'], dtype='object')

In [10]:
# split dataset with features
X = data[feature_name]
y = data['rating']

# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### BaggingClassifier

#### with default parameters

In [11]:
# fitting BaggingClassifier to the dataset
from sklearn.ensemble import BaggingClassifier

bc = BaggingClassifier()
bc.fit(X_train, y_train)

BaggingClassifier()

In [12]:
# model is overfitting training dataset
print('Training Accuracy : %.3f'%bc.score(X_train, y_train))
print('Test Accuracy : %.3f'%bc.score(X_test, y_test))

Training Accuracy : 0.985
Test Accuracy : 0.405


#### with hyperparameter

In [13]:
# GridsearchCV with DecisionTree (default base estimator)
params = {'base_estimator': [None],
         'n_estimators': np.arange(100,400,10)
         }

bc_grid = GridSearchCV(BaggingClassifier(random_state=1), 
                       param_grid = params, 
                       scoring="accuracy", 
                       cv=3, n_jobs=-1, verbose=1)
bc_grid.fit(X_train, y_train)

print(bc_grid.best_params_)
print(bc_grid.best_score_)

Fitting 3 folds for each of 30 candidates, totalling 90 fits
{'base_estimator': None, 'n_estimators': 390}
0.4011971510298374


In [15]:
# Accuracy score on test data is still low
bc = BaggingClassifier(n_estimators=390)
bc.fit(X_train, y_train)

print('Training Accuracy : %.3f'%bc.score(X_train, y_train))
print('Test Accuracy : %.3f'%bc.score(X_test, y_test))

Training Accuracy : 1.000
Test Accuracy : 0.412


#### with hyperparameter on a different base_estimator

In [ ]:
n_samples = data.shape[0]
n_features = data.shape[1]

params = {'base_estimator': [None],
          'n_estimators': [20,50,100],
          'max_samples': [0.5, 1.0, n_samples//2, ],
          'bootstrap': [True, False],
          'bootstrap_features': [True, False]}

bagging_classifier_grid = GridSearchCV(BaggingClassifier(random_state=1, n_jobs=-1), param_grid =params, cv=3, n_jobs=-1, verbose=1)
bagging_classifier_grid.fit(X_train, y_train)

print('Train Accuracy : %.3f'%bagging_classifier_grid.best_estimator_.score(X_train, y_train))
print('Test Accuracy : %.3f'%bagging_classifier_grid.best_estimator_.score(X_test, y_test))
print('Best Accuracy Through Grid Search : %.3f'%bagging_classifier_grid.best_score_)
print('Best Parameters : ',bagging_classifier_grid.best_params_)

In [ ]:
# Accuracy score on test data with hyperparameters
bc = BaggingClassifier({'bootstrap': False, 'bootstrap_features': True, 'max_samples': 0.5, 'n_estimators': 100})
bc.fit(X_train, y_train)

print('Training Accuracy : %.3f'%bc.score(X_train, y_train))
print('Test Accuracy : %.3f'%bc.score(X_test, y_test))

In [14]:
### Comparison metrics

In [ ]:
from sklearn import metrics

metrics.confusion_matrix(y_test,y_pred)
#print precision value of model
print(metrics.precision_score(x_test["y_initial"],x_test["y_pred"]))
#print recall value of model
print(metrics.recall_score(x_test["y_initial"],x_test["y_pred"]))
#print accuracy of model
print(metrics.accuracy_score(x_test["y_initial"],x_test["y_pred"]))
from sklearn.metrics import roc_auc_score
print(metrics.roc_auc_score(x_test["y_initial"],x_test["y_pred"]))

fpr, tpr, thresholds = metrics.roc_curve(x_test["y_initial"],x_test["y_pred"])
roc_auc = metrics.auc(fpr, tpr)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr)
display.plot()
plt.show()

*The model is not fitting the dataset as is, we need to choose a different model.*